In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from pickle import dump
from pickle import load 
from keras.models import load_model
import pickle

Using TensorFlow backend.


In [2]:
# Note: Colab pipeline meant for continuing model training 
# We load the processed character text instead of the ready-to-run X and y data
# because uploading X and y data usually too large to upload. 
# We repeat the preprocessing steps 
with open("process_shakespeare_final.txt", "rb") as fp: 
    char_text = pickle.load(fp)

In [3]:
mapping = load(open("mapping.pkl", "rb"))
vocab_size = len(mapping)

In [4]:
length = 40 + 1
sequences = []
for i in range(length, len(char_text)): 
    seq = char_text[i-length:i]
    seq_string = ''.join(seq)
    sequences.append(seq_string)
print(len(sequences))

91786


In [5]:
num_char = []
for line in sequences: 
    encoded = [mapping[char] for char in line]
    num_char.append(encoded)
print(len(num_char[0]))

41


In [6]:
num_char = np.array(num_char)
X, y = num_char[:, :-1], num_char[:, -1]
print(X.shape)
print(y.shape)

(91786, 40)
(91786,)


In [7]:
X = np.array([to_categorical(x, num_classes=len(mapping)) for x in X])
y = to_categorical(y, num_classes=len(mapping))
print(X.shape)
print(y.shape)

(91786, 40, 60)
(91786, 60)


In [ ]:
# For running locally, you can load directly and then stack 
with open("X_training_shakespeare_final.h5", "rb") as fp: 
    X = pickle.load(fp)
with open("y_training_shakespeare_final.h5", "rb") as fp: 
    y = pickle.load(fp)

In [8]:
# Load from latest version of model 5 to continue training
model = load_model("model5_6.h5")
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 40, 120)           86880     
_________________________________________________________________
dropout_1 (Dropout)          (None, 40, 120)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 120)               115680    
_________________________________________________________________
dense_3 (Dense)              (None, 60)                7260      
Total params: 209,820
Trainable params: 209,820
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs = 1, batch_size = 64, verbose=1)

Epoch 1/1
91786/91786 [==============================] - 245s 3ms/step - loss: 1.0347 - acc: 0.6638


In [ ]:
# Save the trained model
filename = "model5_7.h5"
model.save(filename)

In [ ]:
# Download from colab to local disk
from google.colab import files
files.download(filename)